In [7]:
import pandas as pd
from cuotasimss import CuotasImss
from imss import CalculoIMSSObrero, CalculoIMSSPatron
from isr import ISR
from datetime import datetime
from sbc import SBC
pd.options.display.float_format = '{:,.2f}'.format

In [19]:
df = pd.read_excel('./data/plantilla_laboral2.xlsx', sheet_name='Hoja1') #2022-10-27 incremento
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso
0,1,Martha Rosa Alvarez Martínez,"33,256.68",1500,2019-06-03
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50",1297,2019-06-17
2,6,Karla Berenice Cano Garcia,"22,960.82",1148,2019-07-23
3,7,Jonathan Covarrubias Sustaita,"21,719.58",1086,2019-03-04
4,8,Miguel Dionicio Luna,"27,315.62",1366,2018-01-09


In [20]:
df['Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%Y-%m-%d')

In [21]:
date_2022 = pd.to_datetime('2022/12/31')
date_ = pd.to_datetime('2022/12/31')

In [22]:
df['SD'] = df['Sueldo mensual'].apply(lambda x: x / 30)
df['SBC'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo mensual'], vales = x['Vales'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['ISR'] = (df['Sueldo mensual'] * 1).apply(lambda x: ISR.function_isr(x))
df['IMSS_Ob'] = df['SBC'].apply(lambda x: CalculoIMSSObrero.function_imss_obrero(x))
df['IMSS_Pt'] = df.apply(lambda x: CalculoIMSSPatron.function_imss_patronal(x = x['SBC'], date=date_ ), axis=1)
df['ISN'] = df['Sueldo mensual'] * 0.03
_isr_ = df['Sueldo mensual'].apply(lambda x: ISR.function_isr(x))
df['Neto'] = df['Sueldo mensual'] - _isr_ - df['IMSS_Ob']
df['Aguinaldo'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['Prima Vacacional'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo mensual'], date_ingreso = x['Fecha Ingreso'], date_base = date_2022), axis=1)

In [23]:
df

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SD,SBC,ISR,IMSS_Ob,IMSS_Pt,ISN,Neto,Aguinaldo,Prima Vacacional
0,1,Martha Rosa Alvarez Martínez,"33,256.68",1500,2019-06-03,"1,108.56","1,172.56","5,797.85",954.07,"6,292.64",997.70,"26,504.76","16,628.34","2,771.39"
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50",1297,2019-06-17,864.38,910.00,"4,097.80",732.57,"4,995.59",777.94,"21,101.13","12,965.75","2,160.96"
2,6,Karla Berenice Cano Garcia,"22,960.82",1148,2019-07-23,765.36,802.05,"3,463.27",641.51,"4,462.32",688.82,"18,856.04","11,480.41","1,913.40"
3,7,Jonathan Covarrubias Sustaita,"21,719.58",1086,2019-03-04,723.99,758.70,"3,198.14",604.93,"4,248.15",651.59,"17,916.51","10,859.79","1,809.97"
4,8,Miguel Dionicio Luna,"27,315.62",1366,2018-01-09,910.52,961.87,"4,400.52",776.33,"5,251.82",819.47,"22,138.77","13,657.81","2,731.56"
5,9,José Manuel Franco González,"38,020.36",1500,2015-07-09,"1,267.35","1,342.43","6,918.27","1,097.37","7,131.79","1,140.61","30,004.72","19,010.18","4,435.71"
6,11,Ana Gómez Gallardo Aguilar,"82,484.98",1500,2018-02-01,"2,749.50","2,895.94","19,990.42","1,994.18","12,383.36","2,474.55","60,500.38","41,242.49","8,248.50"
7,12,Juan Manuel González Barajas,"177,157.22",1500,2018-11-05,"5,905.24","6,207.31","51,663.04","1,994.18","12,383.36","5,314.72","123,500.00","88,578.61","17,715.72"
8,13,Gerardo González Flores,"21,719.58",1086,2019-09-02,723.99,758.70,"3,198.14",604.93,"4,248.15",651.59,"17,916.51","10,859.79","1,809.97"
9,14,Mónica González Garduño,"20,481.40",1024,2018-01-24,682.71,716.38,"2,933.66",569.24,"4,039.12",614.44,"16,978.50","10,240.70","2,048.14"


In [ ]:
df.head()

In [ ]:
df.tail(5)

In [ ]:
df.query('Nombre.str.contains("Rogelio")', engine='python')

In [ ]:
calculo = df.query('Nombre.str.contains("Sergio")', engine='python')

In [ ]:
calculo.T

In [18]:
df.to_excel('./resultados/calculos_dic_.xlsx', index=False, sheet_name='2022-12-31')

In [ ]:
df.query('Nombre.str.contains("Ana Gómez")', engine='python').T